In [ ]:
import os
import pandas as pd
import math
import csv

## Function to obtain corrected copy number based on purity and plody estimate ##

In [ ]:
def logr2tumcn(cellularity, total_ploidy, logR):
	return (((total_ploidy*(2**logR)) - 2*(1-cellularity)) / cellularity)

## First obtain purity and ploidy estimates (from a copy number caller such as BATTENBERG) ##

In [ ]:
cellularity = 1
total_ploidy = 2.0
logR = 1.50419 #log2
print(logr2tumcn(cellularity, total_ploidy, logR))

In [ ]:
purity_dict = {} #keep track of sample purity
ploidy_dict = {} #keep track of sample ploidy

#first record Battenberg purity and ploidy
df = pd.read_csv("/data/khandekara2/dev/Sherlock-Lung/sherlock_lung_final_wgs_1217.txt", sep="\t")
for s,n, purity, ploidy in zip(list(df["Tumor_File"]), list(df["Normal_File"]), list(df["Tumor_Purity"]), list(df["Tumor_Ploidy"])):
    if not pd.isna(s) and not pd.isna(n):
        if s.split("/")[-2].startswith("v"):
            sample = s.split("/")[-3]
        else:
            sample=s.split("/")[-2]
        purity_dict[sample] = float(purity)
        ploidy_dict[sample] = float(ploidy)

## Run CNVkit (version 0.9.9, example command for one sample) to segment genome (Circular Binary Segmentation) ##
**Note this has already been done for all samples**

**Results can be found here: /data/Sherlock_Lung/Analysis_results/Azhar/CNVkit**

In [ ]:
# /data/khandekara2/bin/anaconda3/envs/cnvkit/bin/python /data/khandekara2/bin/anaconda3/bin/cnvkit.py batch /data/ITEB_Lung_WGS/FireCloud_downloads/Paper2/NCI_75N_DCEG_NSLC_Shipment1_Batch2_124samples/NSLC-AJOC-TTP1-A-1-1-D-A79I-36/v1/NSLC-AJOC-TTP1-A-1-1-D-A79I-36.cram -m wgs --fasta /data/khandekara2/bin/AmpliconArchitect/data_repo/GRCh38/GCA_000001405.15_GRCh38_no_alt_analysis_set.fa -p 24 -d /data/khandekara2/dev/Sherlock-Lung/results/ecDNA/cnvkit-tumor-normal2/NSLC-AJOC-TTP1-A-1-1-D-A79I-36_cnvkit_output/ --normal /data/ITEB_Lung_WGS/FireCloud_downloads/Paper2/ncicontract_dceg_nslc_wgs_datadelivery_aug2019_619samples/RS122271/v1/RS122271.cram  
# /data/khandekara2/bin/anaconda3/envs/cnvkit/bin/python /data/khandekara2/bin/anaconda3/bin/cnvkit.py segment /data/khandekara2/dev/Sherlock-Lung/results/ecDNA/cnvkit-tumor-normal2/NSLC-AJOC-TTP1-A-1-1-D-A79I-36_cnvkit_output/NSLC-AJOC-TTP1-A-1-1-D-A79I-36.cnr  -p 24 -m cbs -o /data/khandekara2/dev/Sherlock-Lung/results/ecDNA/cnvkit-tumor-normal2/NSLC-AJOC-TTP1-A-1-1-D-A79I-36_cnvkit_output/NSLC-AJOC-TTP1-A-1-1-D-A79I-36.cns

## Rescale CNV calls and obtain corrected TCN estimate and then submit swarm job to run AA pipeline ## 

In [ ]:
df2 = pd.read_csv("/data/khandekara2/dev/Sherlock-Lung/sherlock_lung_final_wgs_1217.txt", sep="\t") #metadata file
mapping = dict(zip(df2["Tumor_Sample_ID"], df2["Study"]))
df2.set_index("Tumor_Sample_ID", inplace = True)


#here you can find the original seeds from CNVkit as well as the "corrected" seeds
os.chdir("/scratch/zhangt8/Azhar/CNVkit")


#INPUTS: logR for a given segment, estimated tumor purity and ploidy for the sample
#OUTPUTS: total copy number for the segment
with open("/scratch/zhangt8/Azhar/AA.rescaled.swarm", "w") as f2:
    #now go through all .cns files from CNVkit (these contain the logR estimates)
	for file in os.listdir("."):
		if file.endswith(".cns"):
			s = file.split(".")[0]
			if s in mapping:
				print(s)
                #write new CNV segment file which contains corrected total copy number
				with open(file, 'r') as f, open(s + ".rescaled.CN.bed", "w") as csvout:
					writer = csv.writer(csvout, delimiter="\t")
					next(f)
					for line in f:
						line = line.rstrip().split("\t")
						log2 = float(line[4])
						if s in purity_dict and s in ploidy_dict:
							cellularity = purity_dict[s]
							total_ploidy = ploidy_dict[s]
							y = logr2tumcn(cellularity, total_ploidy, log2) #rescaled(corrected) total copy number 
							writer.writerow([line[0], line[1], line[2], y])   
						else:
							print(s)
				sample = s
				s = df2.loc[s, "Tumor_File"]
				#print(sample, s)
				f2.write("module load samtools/1.15 && export REF_PATH=/data/Sherlock_Lung/Share/Reference/hg38/cache/%2s/%2s/%s && " + "/data/khandekara2/bin/PrepareAA/PrepareAA.py -s " + sample + " --ref GRCh38 -t 24 --sorted_bam " + s + " --cnv_bed /scratch/zhangt8/Azhar/CNVkit/" + sample + ".rescaled.CN.bed " + "--python3_path /data/khandekara2/bin/anaconda3/envs/cnvkit/bin/python --output_directory /scratch/zhangt8/Azhar/results --downsample 40 --run_AA --run_AC\n")

##### IMPORTANT: Parameters that have been changed from default based on extensive testing: #

* --AA_insert_sdevs 9.0 DEFAULT: 3.0 Rationale: Sequencing libraries with fragment length distributions that have high variance need to have a higher cutoff for standard deviations above/below expected in order to call a structural variant

* --cngain 10  DEFAULT: 4.5 Rationale: Copy number calls that have already been corrected for purity and ploidy need to have a higher CN cutoff to be considered putative ecDNA 

## Here is an example command to run the entire pipeline for one sample ##

In [ ]:
/data/khandekara2/bin/PrepareAA/PrepareAA.py -s SB806762 --ref GRCh38 -t 24 --cnv_bed /scratch/zhangt8/Azhar/CNVkit/SB806762.rescaled.CN.bed --sorted_bam /data/ITEB_Lung_WGS/FireCloud_downloads/Paper3/13Nov_835samples/SB806762/v1/SB806762.cram --python3_path /data/khandekara2/bin/anaconda3/envs/cnvkit/bin/python --output_directory /scratch/zhangt8/Azhar/results --AA_insert_sdevs 9.0 --cngain 10 --run_AA --run_AC 

## Command to submit SWARM job ##

In [ ]:
swarm -f /data/Sherlock_Lung/Analysis_results/AA.rescaled.swarm -g 6 -t 32 --module R/4.0,samtools --logdir logs/ --time 72:00:00

## All output results can be found here ##

*/data/Sherlock_Lung/Analysis_results/Azhar/rescaled5*